In [73]:


import pandas as pd
from sklearn.model_selection import GridSearchCV
from joblib import load
import numpy as np
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import random
import json

In [74]:
city = 'hcm'
model_name = 'xgb'
version = 0
# model = load(f'/home/long/airflow/dags/models/{city}/{model_name}/v{version}/model.joblib')

In [75]:
df = pd.read_parquet(f'/home/long/long/datn-feast/data/update_data/demo1/realestate_dataset_{city}_v{version}.parquet')
df

,target,event_timestamp,realestate_id,typeOfRealEstate,accessibility,ward,certificateOfLandUseRight,houseDirection,street,district,facade,lon,frontWidth,lat,w,numberOfBathRooms,landSize,numberOfBedRooms
0,0.000000,2023-08-23 00:00:00+00:00,0,6,0,219,5000,5000,1089,15,0,106.819703,0.0,10.826577,0.0,4.477301,25.0,4.694865
1,46.875000,2023-09-04 00:00:00+00:00,1,1,0,266,5000,5000,3657,35,0,106.577329,0.0,10.883938,0.0,4.477301,96.0,4.694865
2,41.071429,2023-10-04 00:00:00+00:00,2,0,0,334,5000,5000,590,0,0,106.753847,0.0,10.861560,0.0,1.000000,56.0,2.000000
3,0.000000,2023-10-04 00:00:00+00:00,3,6,0,60,5000,5000,1305,10,0,106.639256,70.0,10.793069,70.0,4.477301,8000.0,4.694865
4,133.333333,2023-10-13 00:00:00+00:00,4,2,0,251,5000,5000,630,18,2,106.761276,0.0,10.777639,0.0,4.477301,300.0,4.694865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,173.913043,2024-07-07 22:13:03+00:00,2096,1,0,130,5000,5000,162,20,0,106.693437,0.0,10.809214,0.0,4.000000,46.0,5.000000
2097,98.529412,2024-07-07 22:13:29+00:00,2097,1,0,175,5000,5000,511,0,0,106.761686,0.0,10.844976,0.0,3.000000,68.0,3.000000
2098,191.428571,2024-07-07 22:15:29+00:00,2098,1,0,137,5000,5000,2494,10,0,106.652596,0.0,10.791150,0.0,5.000000,35.0,4.000000
2099,44.333333,2024-07-07 22:21:27+00:00,2099,1,0,243,5000,5000,5000,0,0,106.724553,0.0,10.871886,0.0,4.477301,150.0,3.000000


In [76]:
data = json.load(open('/home/long/airflow/dags/schema/expectations/address.json', 'r'))

In [77]:
featureset = json.load(open(f'/home/long/long/datn-feast/data/featureset/{city}_v{version}.json'))

In [78]:
order_config = {
    'abr': "num-cat",
    'gbr': "num-cat",
    'etr': 'cat-num',
    'cat': 'cat-num',
    'knr': 'num-cat',
    'linear': 'num-cat',
    'ridge': 'num-cat',
    'mlp': 'num-cat',
    'rf': 'cat-num',
    'xgb': 'cat-num',
    'lgbm': 'cat-num',
    'la': 'num-cat'
}
order = order_config[model_name]

In [79]:
cat_cols = featureset['cat_cols']
num_cols = featureset['num_cols']

In [80]:
if order == "num-cat":
    all_cols = num_cols + cat_cols
else:
    all_cols =  cat_cols + num_cols

In [81]:
X = df[all_cols]
y = df['target']

In [82]:
X

,typeOfRealEstate,accessibility,ward,certificateOfLandUseRight,houseDirection,street,district,facade,lon,frontWidth,lat,w,numberOfBathRooms,landSize,numberOfBedRooms
0,6,0,219,5000,5000,1089,15,0,106.819703,0.0,10.826577,0.0,4.477301,25.0,4.694865
1,1,0,266,5000,5000,3657,35,0,106.577329,0.0,10.883938,0.0,4.477301,96.0,4.694865
2,0,0,334,5000,5000,590,0,0,106.753847,0.0,10.861560,0.0,1.000000,56.0,2.000000
3,6,0,60,5000,5000,1305,10,0,106.639256,70.0,10.793069,70.0,4.477301,8000.0,4.694865
4,2,0,251,5000,5000,630,18,2,106.761276,0.0,10.777639,0.0,4.477301,300.0,4.694865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,1,0,130,5000,5000,162,20,0,106.693437,0.0,10.809214,0.0,4.000000,46.0,5.000000
2097,1,0,175,5000,5000,511,0,0,106.761686,0.0,10.844976,0.0,3.000000,68.0,3.000000
2098,1,0,137,5000,5000,2494,10,0,106.652596,0.0,10.791150,0.0,5.000000,35.0,4.000000
2099,1,0,243,5000,5000,5000,0,0,106.724553,0.0,10.871886,0.0,4.477301,150.0,3.000000


In [83]:
pretrained_file = f'/home/long/airflow/dags/models/{city}/{model_name}/v{version}/model.joblib'

In [84]:
categorical_features_indices = [i for i, c in enumerate(all_cols) if c in cat_cols]

In [85]:
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from joblib import load
import numpy as np
import lightgbm as lgb

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class BKPriceEstimator(RegressorMixin, BaseEstimator):
    def __init__(self, pretrained_model_path, update_model, weight = 0.001, type = None) -> None:
        self.pretrained_model_path = pretrained_model_path
        self.update_model = update_model
        self.weight = weight
        self.type = type

    def fit(self, X, y=None):

        self.X = X
        self.y = y
        print("Start training update model")
        self.update_model.fit(X, y)
        print("End training update model")
        print()
        if self.type == 'lgbm':
            self.pretrained_model = load(self.pretrained_model_path)
            self.pretrained_model.booster_.save_model('./lgbm_cache.txt')
            self.pretrained_model = lgb.Booster(model_file='./lgbm_cache.txt')
        else:
            self.pretrained_model = load(self.pretrained_model_path)
        self.is_fitted_ = True
        return self

    def predict(self, X):
        check_is_fitted(self, ['is_fitted_', 'update_model'])

        base_pred = self.pretrained_model.predict(X)
        base_pred = np.array([max(0, item) for item in base_pred])
        new_pred = self.update_model.predict(X)
        new_pred = np.array([max(0, item) for item in new_pred])


        result = base_pred * (1 - self.weight) + new_pred * self.weight
        # result = base_pred

        return result

    def get_params(self, deep = False):
        return {
            'weight':self.weight,
            'pretrained_model_path':self.pretrained_model_path,
            'update_model':self.update_model,
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self





In [86]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import random

from joblib import load

def create_model(pretrained_file = None):
    update_model = XGBRegressor(n_estimators = 1000, random_state=random.randint(3, 1000), verbose=10, learning_rate=0.04, tree_method = 'gpu_hist')
    model = BKPriceEstimator(
        pretrained_model_path = pretrained_file,
        update_model = update_model,
    )
    search_params = {'weight': [-0.001, 0, 0.001]}
    clf = GridSearchCV(model, search_params, scoring=['explained_variance', 'max_error', 'neg_root_mean_squared_error', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_percentage_error'], refit='neg_root_mean_squared_error', cv=5)
    return clf

model = create_model(pretrained_file)

In [87]:
model.fit(X, y)

Start training update model


/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:20:58] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:20:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


End training update model



/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:21:11] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)
/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:21:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Start training update model


...

Start training update model


/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:25:37] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:25:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


End training update model



/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [08:25:50] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=5,
             estimator=BKPriceEstimator(pretrained_model_path='/home/long/airflow/dags/models/hcm/xgb/v0/model.joblib',
                                        update_model=XGBRegressor(base_score=None,
                                                                  booster=None,
                                                                  callbacks=None,
                                                                  colsample_bylevel=None,
                                                                  colsample_bynode=None,
                                                                  colsample_bytree=None,
                                                                  device=None,
                                                                  early_stopping_rounds=None,
                                                                  enable_categorical=False,
                                                                  eval_metric=None,

In [104]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

In [89]:
test_df = pd.read_parquet(f'/home/long/long/datn-feast/data/data_df_{city}_v{version}.parquet')
target_test_df = pd.read_parquet(f'/home/long/long/datn-feast/data/target_df_{city}_v{version}.parquet')

In [90]:
sample_dict = {
    "hcm": 35000,
    "hn": 100000
}

In [91]:
X_test = test_df[all_cols].iloc[sample_dict[city]:]
y_test = target_test_df['target'].iloc[sample_dict[city]:]

In [92]:
X_test

,typeOfRealEstate,accessibility,ward,certificateOfLandUseRight,houseDirection,street,district,facade,lon,frontWidth,lat,w,numberOfBathRooms,landSize,numberOfBedRooms
35000,1,4,248,1,0,1771,22,0,106.620232,0.0,10.769421,0.0,4.477301,38.0,4.694865
35001,3,1,60,1,0,531,10,2,106.640272,20.0,10.793657,20.0,4.477301,130.0,4.694865
35002,2,1,374,0,4,737,0,0,106.730597,10.0,10.843807,10.0,4.477301,202.0,4.694865
35003,3,1,176,0,0,229,30,2,106.629396,0.0,10.784296,0.0,2.000000,52.0,2.000000
35004,3,0,224,1,0,745,1,2,106.724977,14.0,10.749313,14.0,4.477301,308.0,4.694865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44144,3,1,140,1,4,1653,30,2,106.625024,4.0,10.815720,4.0,4.477301,100.0,4.694865
44145,1,0,176,1,0,229,30,0,106.629396,0.0,10.784296,0.0,4.477301,50.1,4.000000
44146,1,0,278,1,0,934,11,0,106.689453,0.0,10.761833,0.0,20.000000,480.0,20.000000
44147,3,0,391,1,0,1202,42,2,106.749881,0.0,10.684462,0.0,4.477301,90.0,4.000000


In [95]:
cv_results = model.cv_results_
best_index = model.best_index_

In [96]:
for score_name in [score for score in cv_results if "mean_test" in score]:
    print(score_name, cv_results[score_name][best_index])
    print(score_name.replace("mean","std", 1), cv_results[score_name.replace("mean","std", 1)][best_index])

mean_test_explained_variance 0.5294729569995094
std_test_explained_variance 0.24097000908459823
mean_test_max_error -693.7950563646868
std_test_max_error 582.3771371301547
mean_test_neg_root_mean_squared_error -68.76214994993995
std_test_neg_root_mean_squared_error 30.488945188871124
mean_test_r2 0.5162847972510276
std_test_r2 0.250865827822769
mean_test_neg_median_absolute_error -17.617489272702784
std_test_neg_median_absolute_error 3.204725831188236
mean_test_neg_mean_absolute_percentage_error -2.394241355476502e+16
std_test_neg_mean_absolute_percentage_error 3.0776817728132864e+16


In [105]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred, squared = False)
rsq = r2_score(y_test, y_pred)
ev_score = explained_variance_score(y_test, y_pred)
print(f'mae {mae:.5f}, rmse {mse:.5f}, rsq {rsq:.5f}', ev_score)

mae 37.48820, rmse 62.09203, rsq 0.81066 0.8110394216423011


In [122]:
list(model.best_estimator_.pretrained_model.feature_importances_)

array([0.4138727 , 0.00709314, 0.01525556, 0.00831323, 0.00821211,
       0.01536901, 0.07718424, 0.05991684, 0.12416454, 0.01081986,
       0.183482  , 0.        , 0.02587353, 0.02263431, 0.02780884],
      dtype=float32)

In [124]:
weights = list(model.best_estimator_.pretrained_model.feature_importances_)
print(len(weights))
feature_important_df = pd.DataFrame()
feature_important_df['features'] = all_cols
feature_important_df['importance_scale_weight'] = weights
feature_important_df = feature_important_df.sort_values(by = ['importance_scale_weight'], ascending=False)
feature_important_df

15


,features,importance_scale_weight
0,typeOfRealEstate,0.413873
10,lat,0.183482
8,lon,0.124165
6,district,0.077184
7,facade,0.059917
14,numberOfBedRooms,0.027809
12,numberOfBathRooms,0.025874
13,landSize,0.022634
5,street,0.015369
2,ward,0.015256
